In [1]:
import os
import importlib
from utility import Pi_util
from natsort import natsorted
import numpy as np
import pickle
import sys
import pandas as pd

path_to_lib: str = "/Users/zacharykelly/Documents/MATLAB/projects/combiExperiments/code/lightLogger/miniSpect"
sys.path.append(path_to_lib)
import MS_util
import importlib

In [2]:
experiment_path: str = "/Volumes/EXTERNAL1/fixBufferIssue_0.1hz_0NDF"
use_mean_frame: bool = True

In [3]:
Pi_util.parse_chunks_pkl(experiment_path, use_mean_frame)

Loading chunk: 1/90
Loading chunk: 2/90
Loading chunk: 3/90
Loading chunk: 4/90
Loading chunk: 5/90
Loading chunk: 6/90
Loading chunk: 7/90
Loading chunk: 8/90
Loading chunk: 9/90
Loading chunk: 10/90
Loading chunk: 11/90
Loading chunk: 12/90
Loading chunk: 13/90
Loading chunk: 14/90
Loading chunk: 15/90
Loading chunk: 16/90
Loading chunk: 17/90
Loading chunk: 18/90
Loading chunk: 19/90
Loading chunk: 20/90
Loading chunk: 21/90
Loading chunk: 22/90
Loading chunk: 23/90
Loading chunk: 24/90
Loading chunk: 25/90
Loading chunk: 26/90
Loading chunk: 27/90
Loading chunk: 28/90
Loading chunk: 29/90
Loading chunk: 30/90
Loading chunk: 31/90
Loading chunk: 32/90
Loading chunk: 33/90
Loading chunk: 34/90
Loading chunk: 35/90
Loading chunk: 36/90
Loading chunk: 37/90
Loading chunk: 38/90
Loading chunk: 39/90
Loading chunk: 40/90
Loading chunk: 41/90
Loading chunk: 42/90
Loading chunk: 43/90
Loading chunk: 44/90
Loading chunk: 45/90
Loading chunk: 46/90
Loading chunk: 47/90
Loading chunk: 48/90
L

ValueError: buffer size must be a multiple of element size